In [1]:
import tensorflow as tf
from tensorflow.python.keras.applications import resnet50

from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, ZeroPadding2D
from tensorflow.python.keras.models import Sequential, Model

from tensorflow.python.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [2]:
img_width, img_height, img_channel = 224, 224, 3

In [3]:
base_model = resnet50.ResNet50(include_top = False, input_shape=(img_width, img_height, img_channel))

In [4]:
nb_classes = 5

In [5]:
base_model.outputs = [base_model.layers[-1].output]

In [6]:
last = base_model.outputs[0]
x = GlobalAveragePooling2D()(last)
preds = Dense(nb_classes, activation='softmax')(x)

In [7]:
tl_model = Model(base_model.input, preds)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
tl_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# es = EarlyStopping(patience=10, monitor='val_acc')
tb = TensorBoard(log_dir='resnet_finetune')
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
cp = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [11]:
train_data_dir = '/input/train'
validation_data_dir = '/input/validation'
test_data_dir = '/input/test'

In [12]:
datagen = ImageDataGenerator(rescale=1./255, 
                             horizontal_flip=True,
                             width_shift_range=0.2, 
                             height_shift_range=0.2,
                             shear_range=0.02)

test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
tr_batchsize = 32
val_batchsize = 32
test_batchsize = 32

In [14]:
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=tr_batchsize,
    class_mode='categorical')


validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=val_batchsize,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=test_batchsize,
    class_mode='categorical')

Found 1483 images belonging to 5 classes.
Found 184 images belonging to 5 classes.
Found 154 images belonging to 5 classes.


In [15]:
tl_model.fit_generator(generator=train_generator, 
                       epochs=50, 
                       steps_per_epoch=train_generator.n // tr_batchsize, 
                       validation_data=validation_generator, 
                       validation_steps=validation_generator.n // val_batchsize,
                       callbacks=[cp, tb])

Epoch 1/50
46/46 [==============================] - 21s - loss: 1.3925 - acc: 0.4062 - val_loss: 1.9407 - val_acc: 0.2313
Epoch 2/50
46/46 [==============================] - 16s - loss: 0.9766 - acc: 0.6330 - val_loss: 1.8338 - val_acc: 0.2375
Epoch 3/50
46/46 [==============================] - 16s - loss: 0.8084 - acc: 0.7222 - val_loss: 2.0456 - val_acc: 0.2125
Epoch 4/50
46/46 [==============================] - 17s - loss: 0.7213 - acc: 0.7549 - val_loss: 2.3953 - val_acc: 0.1812
Epoch 5/50
46/46 [==============================] - 17s - loss: 0.6745 - acc: 0.7599 - val_loss: 2.6731 - val_acc: 0.1938
Epoch 6/50
46/46 [==============================] - 16s - loss: 0.6233 - acc: 0.7962 - val_loss: 2.8329 - val_acc: 0.2000
Epoch 7/50
46/46 [==============================] - 17s - loss: 0.5964 - acc: 0.7937 - val_loss: 2.9729 - val_acc: 0.1875
Epoch 8/50
46/46 [==============================] - 16s - loss: 0.5579 - acc: 0.8006 - val_loss: 3.1034 - val_acc: 0.1688
Epoch 9/50
46/46 [======

46/46 [==============================] - 16s - loss: 0.3003 - acc: 0.9071 - val_loss: 0.5680 - val_acc: 0.7812
Epoch 31/50
46/46 [==============================] - 17s - loss: 0.2951 - acc: 0.9057 - val_loss: 0.4977 - val_acc: 0.8250
Epoch 32/50
46/46 [==============================] - 16s - loss: 0.2714 - acc: 0.9151 - val_loss: 0.6129 - val_acc: 0.8063
Epoch 33/50
46/46 [==============================] - 17s - loss: 0.2982 - acc: 0.9050 - val_loss: 0.6091 - val_acc: 0.7688
Epoch 34/50
46/46 [==============================] - 17s - loss: 0.2665 - acc: 0.9084 - val_loss: 0.5494 - val_acc: 0.8125
Epoch 35/50
46/46 [==============================] - 17s - loss: 0.2834 - acc: 0.9096 - val_loss: 0.5135 - val_acc: 0.8313
Epoch 36/50
46/46 [==============================] - 16s - loss: 0.2593 - acc: 0.9199 - val_loss: 0.5324 - val_acc: 0.7937
Epoch 37/50
46/46 [==============================] - 16s - loss: 0.2883 - acc: 0.9036 - val_loss: 0.5997 - val_acc: 0.8000
Epoch 38/50
46/46 [=========

In [16]:
tl_model.load_weights("weights-improvement-48-0.87.hdf5")

In [17]:
tl_model.evaluate_generator(generator=test_generator, 
                            steps=test_generator.n // test_batchsize)

[0.3167608380317688, 0.890625]

In [18]:
model_json = tl_model.to_json()

In [19]:
with open("resnet_CAM_new.json", "w") as json_file:
    json_file.write(model_json)

In [20]:
tl_model.save_weights("resnet_CAM_new.h5")